In [1]:
import pandas as pd
import numpy as np 
import json

!pip3 install torch
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

!pip3 install nltk
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer

import random

In [2]:
with open('dataset.json') as t:
    dftext = json.load(t)
print(dftext)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Hello,Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?', 'Welcome to ticket booking service']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.', 'Have a good day', 'Have a great rest of the day', 'Have a great rest of the evening']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', 'Is there anything else I can help you with?']}, {'tag': 'ticket type', 'patterns': ['What type of tickets are available?', 'How many gold tickets are available?', 'How many platinum tickets are available?', 'How many silver tickets are available?', 'How many gold seats are there?', 'How many silver seats are 

In [3]:
all_wds = []
tgs = []
x_y = []

In [4]:

tokenizer = TreebankWordTokenizer()
stemmer = PorterStemmer()

for intt in dftext['intents']:
    tg = intt['tag']
    tgs.append(tg)
    for patrn in intt['patterns']:
        wtk = tokenizer.tokenize(patrn)
        all_wds.extend(wtk)
        x_y.append((wtk, tg))

In [5]:
ignore_syntax = ["?","/",".","!", "@", "#", "$"]
all_wds = [stemmer.stem(wtk) for wtk in all_wds if wtk not in ignore_syntax]

all_wds = sorted(set(all_wds))
tgs = sorted(set(tgs))
print(len(x_y), "patterns")
print(len(tgs), "tags:", tgs)
print(len(all_wds), "Stemmed unique words:", all_wds)

42 patterns
10 tags: ['error', 'goodbye', 'greeting', 'movie type', 'payments', 'thanks', 'ticket type', 'tickets', 'time', 'time slot']
86 Stemmed unique words: ["'s", ',', '100', '2', 'a', 'accept', 'after', 'ambul', 'anyon', 'app', 'are', 'avail', 'be', 'book', 'but', 'bye', 'can', 'card', 'cash', 'complet', 'contractor', 'crash', 'credit', 'day', 'discount', 'do', 'figur', 'for', 'gold', 'good', 'goodby', 'ha', 'havent', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'imposs', 'is', 'k.g.f', 'later', 'lot', 'mani', 'mastercard', 'mission', 'moonfal', 'movi', 'my', 'not', 'of', 'onli', 'option', 'out', 'payment', 'platinum', 'pound', 'price', 'reach', 'reciev', 'rrr', 'seat', 'see', 'should', 'shown', 'silver', 'slot', 'spiderman', 'start', 'student', 'take', 'thank', 'that', 'the', 'there', 'ticket', 'time', 'to', 'type', 'way', 'what', 'which', 'with', 'yet', 'you']


In [6]:
def Bg_Wrds(token_sntce, wrds):
    sntce_wrds = [stemmer.stem(word) for word in token_sntce]

    Bg = np.zeros(len(wrds), dtype=np.float32)
    for indx, r in enumerate(wrds):
        if r in sntce_wrds: 
            Bg[indx] = 1

    return Bg

In [7]:

X_training = []
y_training = []
for (patrn_sntce, tg) in x_y:
    
    Bg = Bg_Wrds(patrn_sntce, all_wds)
    X_training.append(Bg)
    lbl = tgs.index(tg)
    y_training.append(lbl)

X_training =np.array(X_training)
y_training =np.array(y_training)

In [8]:
numepochs = 1000
batch_size = 8
Rate_of_learning = 0.001
size_inputs = len(X_training[0])
hide_size = 8
size_of_output = len(tgs)
print(size_inputs, size_of_output)

86 10


In [14]:
class NNet(nn.Module):
    def __init__(self, size_inputs, hide_size, numclass):
        super(NNet, self).__init__()
        self.l1 = nn.Linear(size_inputs, hide_size) 
        self.l2 = nn.Linear(hide_size, hide_size) 
        self.l3 = nn.Linear(hide_size, numclass)
        self.relu = nn.ReLU()
    
    def forward(self, a):
        outpt = self.l1(a)
        outpt = self.relu(outpt)
        outpt = self.l2(outpt)
        outpt = self.relu(outpt)
        outpt = self.l3(outpt)
        return outpt

In [15]:
class ChatDatas(Dataset):

    def __init__(self):
        self.n_samples = len(X_training)
        self.x_datas = X_training
        self.y_datas = y_training

    def __getitem__(self, indx):
        return self.x_datas[indx], self.y_datas[indx]

    
    def __len__(self):
        return self.n_samples

dataset = ChatDatas()
training_load = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

devices = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model1 = NNet(size_inputs, hide_size, size_of_output).to(devices)


criteria = nn.CrossEntropyLoss()
optimize = torch.optim.Adam(model1.parameters(), lr=Rate_of_learning)


for epoch in range(numepochs):
    for (word, label) in training_load:
        words = word.to(devices)
        label = label.to(dtype=torch.long).to(devices)
        
    
        otputs = model1(words)
        losses = criteria(otputs, label)
        
        optimize.zero_grad()
        losses.backward()
        optimize.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{numepochs}], Loss: {losses.item():.4f}')


print(f'final loss: {losses.item():.4f}')

data = {
"model_state": model1.state_dict(),
"input_size": size_inputs,
"hidden_size": hide_size,
"size_of_output": size_of_output,
"all_words": all_wds,
"tgs": tgs
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [100/1000], Loss: 0.3212
Epoch [200/1000], Loss: 0.1134
Epoch [300/1000], Loss: 0.0422
Epoch [400/1000], Loss: 0.0082
Epoch [500/1000], Loss: 0.0128
Epoch [600/1000], Loss: 0.0012
Epoch [700/1000], Loss: 0.0003
Epoch [800/1000], Loss: 0.0001
Epoch [900/1000], Loss: 0.0001
Epoch [1000/1000], Loss: 0.0004
final loss: 0.0004
training complete. file saved to data.pth


In [16]:
with open('dataset.json', 'r') as jason_datas:
    intents = json.load(jason_datas)

FILE = "data.pth"
data = torch.load(FILE)

size_inputs = data["input_size"]
hide_size = data["hidden_size"]
size_of_output = data["size_of_output"]
all_wds = data['all_words']
tgs = data['tgs']
model_state = data["model_state"]

model = NNet(size_inputs, hide_size, size_of_output
)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentnc = input("You: ")
    if sentnc == "quit":
        break

    sentnc = tokenizer.tokenize(sentnc)
    X = Bg_Wrds(sentnc, all_wds)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    outpt = model(X)
    _, predicted = torch.max(outpt, dim=1)

    tag = tgs[predicted.item()]

    probs = torch.softmax(outpt, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")
        break

Let's chat! (type 'quit' to exit)


KeyboardInterrupt: Interrupted by user